In [1]:
from sampo.generator.base import SimpleSynthetic
from sampo.generator.types import SyntheticGraphType

# 1. Graph generation

In [2]:
# SimpleSynthetic object used for the simple work graph structure generation

r_seed = 231
ss = SimpleSynthetic(r_seed)

In [3]:
# simple graph
# should generate general (average) type of graph with 10 clusters from 100 to 200 vertices each

simple_wg = ss.work_graph(mode=SyntheticGraphType.GENERAL,
                          cluster_counts=10,
                          bottom_border=100,
                          top_border=200)

In [4]:
# complex graph
# should generate general (average) type of graph with 300 unique works, 100 resources and 2000 vertices

advanced_wg = ss.advanced_work_graph(works_count_top_border=2000,
                                     uniq_works=300,
                                     uniq_resources=100)

# 2. Contractor generation

In [5]:
from uuid import uuid4
from sampo.schemas.resources import Worker
from sampo.schemas.contractor import Contractor

### Manual generation
To create contractor, you should provide minimal info: unique id, contractor name, and supplied workers (simple renewable resources).

In [6]:
contractors = [
    Contractor(id=str(uuid4()),
               name="OOO Berezka",
               workers={'worker' : Worker(id='0', name='worker', count=100)})
]

### Generation from graph

In [7]:
from sampo.generator.environment.contractor_by_wg import get_contractor_by_wg

contractors = [get_contractor_by_wg(simple_wg)]

# 3. Scheduling

### Scheduler construction
Before scheduling you should specify scheduling algorithm used for transforming input data to the final schedule.
At this time SAMPO contains heuristic algorithms, such as HEFTAddEnd, HEFTAddBetween and Topological scheduler, and the Genetic algorithm.
While creation, you can specify the hyperparameters to fit the algorithm.

In [8]:
from sampo.scheduler.heft.base import HEFTScheduler

# here we can just create simple heuristic scheduler
scheduler = HEFTScheduler()

In [9]:
from sampo.scheduler.genetic.base import GeneticScheduler

# or more complex genetic scheduler
scheduler = GeneticScheduler(mutate_order=0.05,
                             mutate_resources=0.05)

Can not find native module; switching to default


### Scheduling process
SAMPO provides a simple interface to all its features.
It called SchedulingPipeline.
Using it you only should pass all the scheduling arguments, it remembers them, and you can produce schedules in many ways.

In [10]:
from sampo.pipeline import SchedulingPipeline

project = SchedulingPipeline.create() \
    .wg(simple_wg) \
    .contractors(contractors) \
    .schedule(scheduler) \
    .finish()[0]

project.schedule.execution_time

Genetic optimizing took 30.315876007080078 ms
Toolbox initialization & first population took 5215.619802474976 ms
First population evaluation took 4028.813600540161 ms
-- Generation 1, population=50, best fitness=1120.0 --
-- Generation 2, population=50, best fitness=1068.0 --
-- Generation 3, population=50, best fitness=1016.0 --
-- Generation 4, population=50, best fitness=1016.0 --
-- Generation 5, population=50, best fitness=996.0 --
-- Generation 6, population=50, best fitness=996.0 --
-- Generation 7, population=50, best fitness=996.0 --
-- Generation 8, population=50, best fitness=996.0 --
-- Generation 9, population=50, best fitness=996.0 --
-- Generation 10, population=50, best fitness=996.0 --
-- Generation 11, population=50, best fitness=996.0 --
-- Generation 12, population=50, best fitness=996.0 --
-- Generation 13, population=50, best fitness=996.0 --
-- Generation 14, population=50, best fitness=979.0 --
-- Generation 15, population=50, best fitness=979.0 --
-- Generatio

951

### Other metrics
Genetic scheduler can do more than just optimize MakeSpan. It supports more metrics.
Metrics are represented by `FitnessFunction` class and can be passed to `GeneticScheduler` on constructing.

Here we're constructing Genetic that should optimize resources to deadline.

In [11]:
from sampo.schemas.time import Time
from sampo.scheduler.genetic.operators import DeadlineResourcesFitness

deadline = Time(2000)
# calling `prepare` method to pass explicit parameters to fitness function
fitness_constructor = DeadlineResourcesFitness(deadline)

scheduler = GeneticScheduler(mutate_order=0.05,
                             mutate_resources=0.05,
                             fitness_constructor=fitness_constructor)
scheduler.set_deadline(deadline)

project = SchedulingPipeline.create() \
    .wg(simple_wg) \
    .contractors(contractors) \
    .schedule(scheduler) \
    .finish()[0]

project.schedule.execution_time

Genetic optimizing took 32.92560577392578 ms
Toolbox initialization & first population took 5090.832233428955 ms
First population evaluation took 4190.772771835327 ms
-- Generation 1, population=50, best fitness=334.0 --
-- Generation 1, population=50, best peak=334.0 --
-- Generation 2, population=50, best peak=243.0 --
-- Generation 3, population=50, best peak=243.0 --
-- Generation 4, population=50, best peak=243.0 --
-- Generation 5, population=50, best peak=243.0 --
-- Generation 6, population=50, best peak=243.0 --
-- Generation 7, population=50, best peak=243.0 --
-- Generation 8, population=50, best peak=243.0 --
-- Generation 9, population=50, best peak=243.0 --
-- Generation 10, population=50, best peak=243.0 --
-- Generation 11, population=50, best peak=243.0 --
-- Generation 12, population=50, best peak=243.0 --
-- Generation 13, population=50, best peak=229.0 --
-- Generation 14, population=50, best peak=229.0 --
-- Generation 15, population=50, best peak=229.0 --
-- Gener

1756

Additionally, you can construct other metrics: deadline cost, time with resources (without deadline) fitness:

In [12]:
from sampo.scheduler.genetic.operators import DeadlineCostFitness

fitness_constructor = DeadlineCostFitness(deadline)

scheduler = GeneticScheduler(mutate_order=0.05,
                             mutate_resources=0.05,
                             fitness_constructor=fitness_constructor)
scheduler.set_deadline(deadline)

project = SchedulingPipeline.create() \
    .wg(simple_wg) \
    .contractors(contractors) \
    .schedule(scheduler) \
    .finish()[0]

project.schedule.execution_time

Genetic optimizing took 32.875776290893555 ms
Toolbox initialization & first population took 5288.079023361206 ms
First population evaluation took 4078.0670642852783 ms
-- Generation 1, population=50, best fitness=1375560.0 --
-- Generation 2, population=50, best fitness=1375560.0 --
-- Generation 3, population=50, best fitness=1375560.0 --
-- Generation 4, population=50, best fitness=1375560.0 --
-- Generation 5, population=50, best fitness=1375560.0 --
-- Generation 6, population=50, best fitness=1375560.0 --
-- Generation 7, population=50, best fitness=1375560.0 --
-- Generation 8, population=50, best fitness=1375560.0 --
-- Generation 9, population=50, best fitness=1375560.0 --
-- Generation 10, population=50, best fitness=1375560.0 --
-- Generation 11, population=50, best fitness=1375560.0 --
-- Generation 12, population=50, best fitness=1375560.0 --
Deadline not reached !!! Deadline 2000 < best time 1375560.0
Final time: 1375560.0
Generations processing took 49289.43705558777 ms


1737

In [13]:
from sampo.scheduler.genetic.operators import TimeWithResourcesFitness

fitness_constructor = TimeWithResourcesFitness()

scheduler = GeneticScheduler(mutate_order=0.05,
                             mutate_resources=0.05,
                             fitness_constructor=fitness_constructor)
scheduler.set_deadline(deadline)

project = SchedulingPipeline.create() \
    .wg(simple_wg) \
    .contractors(contractors) \
    .schedule(scheduler) \
    .finish()[0]

project.schedule.execution_time

Genetic optimizing took 30.807018280029297 ms
Toolbox initialization & first population took 4955.848932266235 ms
First population evaluation took 4028.3138751983643 ms
-- Generation 1, population=50, best fitness=1523.0 --
-- Generation 1, population=50, best peak=341.0 --
-- Generation 2, population=50, best peak=307.0 --
-- Generation 3, population=50, best peak=279.0 --
-- Generation 4, population=50, best peak=269.0 --
-- Generation 5, population=50, best peak=269.0 --
-- Generation 6, population=50, best peak=269.0 --
-- Generation 7, population=50, best peak=269.0 --
-- Generation 8, population=50, best peak=269.0 --
-- Generation 9, population=50, best peak=269.0 --
-- Generation 10, population=50, best peak=269.0 --
-- Generation 11, population=50, best peak=269.0 --
-- Generation 12, population=50, best peak=269.0 --
-- Generation 13, population=50, best peak=269.0 --
-- Generation 14, population=50, best peak=269.0 --
-- Generation 15, population=50, best peak=269.0 --
-- Ge

1441